In [26]:
# from the provided file:
    # Add the data to a database
    # seperate the data into the following:
        # first_name
        # last_name
        # middle_name
        # begin_term
        # end_term
        # presedential order

    # Create a report to as a new file with the following in presedential order:
        # order text, 
        # last_name text, 
        # first_name text, 
        # middle_initial text, 
        # begin_term text, 
        # end_term text, 
        # years in office real,

In [ ]:
import sqlite3

file = open("US_Presidents_unordered.txt", 'r')
data = [list(line.split(',')) for line in file.readlines()]
file.close()
data.pop(0)

# removing unwanted characters
for item in data:
    item += item.pop(0).split(' ')
    item[0] = item[0].strip('\'')
    item[1] = item[1].strip() # remove newline character
    item[1] = item[1].replace('\'','').replace('\'','')
    item[2] = item[2].replace('.','').replace('\'','')
    try:
        item[4] = item[4].strip('\'')
        item[5] = item[5].strip('\'')
    except:
        pass
    item.insert(0,item.pop(2))

# Create dictionary with the president number as the key
data = {int(item[0]) : tuple(item[1:2]+item[2:]) for item in data}

# Sort dictionary by it's items index, then convert back to a dictionary
data = dict(sorted(data.items()))

# Build a list of strings to write to the file
data_to_write = [str(index)+ " " + ' '.join(data[index]) for index in data]
data_to_write = [item.split() for item in data_to_write]

# Add a blank value if a presidents middle name does not exist
for item in data_to_write:
    if len(item) < 6:
        item.insert(4, 'n/a')
        
# Add the presidents years in office to the end of each line
[item.append(str(int(item[2][-4:]) - int(item[1][-4:])))for item in data_to_write]


In [478]:
# Open the file in write mode (will be created if it doesn't exist)
file = open("solution.txt", 'w')

# Format the list to a writeable string
[file.write(" ".join(line)+'\n') for line in data_to_write]
file.close()

In [479]:
# Save that data to the database
connection = sqlite3.connect('presidents.db')

connection.execute('''create table if not exists presidents (
        order_number text, 
        last_name text, 
        first_name text, 
        middle_initial text, 
        begin_term text, 
        end_term text, 
        years_in_office real
)''')
[connection.execute('insert into presidents values(?,?,?,?,?,?,?)',(data_to_write[key])) for key,_ in enumerate(data_to_write)]
connection.commit()
connection.close()